# Credit Classification with Quantum Machine Learning

This notebook demonstrates variational quantum classifiers (VQC) and quantum kernel methods for credit risk classification.

**Goal:** Classify loan applicants as creditworthy or risky using quantum ML

**Author:** Ian Buckley  
**Date:** 2025

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
import time

import pennylane as qml
from pennylane import numpy as pnp
from pennylane.optimize import AdamOptimizer

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Imports successful")

## Parameters

In [ ]:
# Data parameters
n_samples = 200
n_features = 4
test_size = 0.3
random_state = 42

# Quantum parameters
n_qubits = n_features
n_layers = 2

# Training parameters
n_epochs = 50
batch_size = 25
learning_rate = 0.01

np.random.seed(random_state)

print(f"Dataset: {n_samples} samples, {n_features} features")
print(f"VQC: {n_qubits} qubits, {n_layers} layers")
print(f"Training: {n_epochs} epochs, batch size {batch_size}")

## Generate Credit Data

Features represent:
- Income level
- Debt-to-income ratio
- Credit history length
- Employment years

In [ ]:
def generate_credit_data(n_samples, n_features, random_state=42):
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_features,
        n_redundant=0,
        n_clusters_per_class=1,
        flip_y=0.1,
        random_state=random_state
    )
    y_quantum = 2 * y - 1  # Convert to -1, +1
    feature_names = ['income', 'debt_ratio', 'credit_history', 'employment_years']
    return X, y, y_quantum, feature_names

X, y, y_quantum, feature_names = generate_credit_data(n_samples, n_features, random_state)

# Split and normalize
X_train, X_test, y_train, y_test, y_train_q, y_test_q = train_test_split(
    X, y, y_quantum, test_size=test_size, random_state=random_state, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Scale to [0, π] for quantum encoding
X_train_scaled = np.pi * (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_test_scaled = np.pi * (X_test - X_test.min()) / (X_test.max() - X_test.min())

print(f"\nTraining samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Class balance: {np.sum(y_train==1)}/{np.sum(y_train==0)} (creditworthy/risky)")

## Visualize Data

In [ ]:
# PCA projection for visualization
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
scatter = plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], 
                     c=y_train, cmap='RdYlGn', alpha=0.6, s=50)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
plt.title('Training Data (PCA Projection)')
plt.colorbar(scatter, label='Class', ticks=[0, 1])
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.bar(range(n_features), pca.components_[0], alpha=0.7, label='PC1')
plt.bar(range(n_features), pca.components_[1], alpha=0.7, label='PC2')
plt.xticks(range(n_features), feature_names, rotation=45, ha='right')
plt.ylabel('Component Loading')
plt.title('Principal Component Loadings')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Variance explained: {sum(pca.explained_variance_ratio_):.1%}")

## Classical SVM Baseline

In [ ]:
print("Training Classical SVM...\n")

start_time = time.time()
svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=random_state)
svm.fit(X_train, y_train)
classical_time = time.time() - start_time

y_train_pred = svm.predict(X_train)
y_test_pred = svm.predict(X_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Training accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")
print(f"Training time: {classical_time:.2f}s\n")

print("Classification Report:")
print(classification_report(y_test, y_test_pred, 
                           target_names=['Risky', 'Creditworthy']))

cm_classical = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:")
print(cm_classical)

## Quantum Feature Map

Encodes classical data into quantum states with entanglement

In [ ]:
def quantum_feature_map(x, wires):
    """ZZ feature map with entanglement."""
    n = len(wires)
    
    # First layer
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(x[i], wires=i)
    
    # Entangling layer
    for i in range(n):
        for j in range(i+1, n):
            qml.CNOT(wires=[wires[i], wires[j]])
            qml.RZ(x[i] * x[j], wires=wires[j])
            qml.CNOT(wires=[wires[i], wires[j]])

print("✓ Quantum feature map defined")

## Variational Quantum Classifier

Architecture:
1. Feature map (encodes data)
2. Variational layers (trainable)
3. Measurement (output)

In [ ]:
def variational_layer(params, wires):
    """Single layer of variational circuit."""
    n = len(wires)
    
    for i, wire in enumerate(wires):
        qml.RY(params[i, 0], wires=wire)
        qml.RZ(params[i, 1], wires=wire)
    
    for i in range(n-1):
        qml.CNOT(wires=[wires[i], wires[i+1]])
    qml.CNOT(wires=[wires[n-1], wires[0]])

dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def vqc_circuit(x, params):
    """Variational quantum classifier circuit."""
    quantum_feature_map(x, wires=range(n_qubits))
    
    for layer in range(n_layers):
        variational_layer(params[layer], wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

print(f"✓ VQC circuit defined")
print(f"  Qubits: {n_qubits}")
print(f"  Layers: {n_layers}")
print(f"  Parameters: {n_layers * n_qubits * 2}")

## Train VQC

In [ ]:
# Initialize parameters
np.random.seed(random_state)
params = np.random.uniform(0, 2*np.pi, (n_layers, n_qubits, 2), requires_grad=True)

optimizer = AdamOptimizer(learning_rate)

train_losses = []
train_accs = []
test_accs = []

print("\nTraining VQC...\n")
print("Epoch  Train Loss  Train Acc  Test Acc   Time")
print("-" * 60)

start_time = time.time()

for epoch in range(n_epochs):
    epoch_start = time.time()
    
    # Shuffle
    indices = np.random.permutation(len(X_train_scaled))
    X_train_shuffled = X_train_scaled[indices]
    y_train_shuffled = y_train_q[indices]
    
    # Mini-batch training
    epoch_loss = 0
    for i in range(0, len(X_train_scaled), batch_size):
        X_batch = X_train_shuffled[i:i+batch_size]
        y_batch = y_train_shuffled[i:i+batch_size]
        
        def cost_fn(params):
            predictions = np.array([vqc_circuit(x, params) for x in X_batch])
            loss = np.mean((y_batch - predictions)**2)
            return loss
        
        params, batch_loss = optimizer.step_and_cost(cost_fn, params)
        epoch_loss += batch_loss
    
    epoch_loss /= (len(X_train_scaled) // batch_size)
    
    # Evaluate
    if (epoch + 1) % 10 == 0 or epoch == 0:
        train_predictions = np.array([vqc_circuit(x, params) for x in X_train_scaled])
        train_predictions_binary = np.sign(train_predictions)
        train_acc = accuracy_score(y_train_q, train_predictions_binary)
        
        test_predictions = np.array([vqc_circuit(x, params) for x in X_test_scaled])
        test_predictions_binary = np.sign(test_predictions)
        test_acc = accuracy_score(y_test_q, test_predictions_binary)
        
        epoch_time = time.time() - epoch_start
        
        print(f"{epoch+1:4d}   {epoch_loss:.4f}      {train_acc:.3f}      {test_acc:.3f}     {epoch_time:.1f}s")
        
        train_losses.append(epoch_loss)
        train_accs.append(train_acc)
        test_accs.append(test_acc)

vqc_time = time.time() - start_time

print(f"\n✓ Training complete in {vqc_time:.1f}s")

## Evaluate VQC

In [ ]:
test_predictions = np.array([vqc_circuit(x, params) for x in X_test_scaled])
test_predictions_binary = np.sign(test_predictions)
vqc_test_acc = accuracy_score(y_test_q, test_predictions_binary)

y_test_01 = (y_test_q + 1) // 2
test_pred_01 = (test_predictions_binary + 1) // 2

print(f"\nFinal Test Accuracy: {vqc_test_acc:.3f}\n")
print("Classification Report:")
print(classification_report(y_test_01, test_pred_01,
                           target_names=['Risky', 'Creditworthy']))

cm_vqc = confusion_matrix(y_test_01, test_pred_01)
print("Confusion Matrix:")
print(cm_vqc)

## Quantum Kernel SVM

Alternative approach: Use quantum feature map to define kernel, then use classical SVM

In [ ]:
def quantum_kernel(x1, x2, n_qubits):
    """Compute quantum kernel between two data points."""
    dev_kernel = qml.device('default.qubit', wires=n_qubits)
    
    @qml.qnode(dev_kernel)
    def kernel_circuit(x1, x2):
        quantum_feature_map(x1, wires=range(n_qubits))
        qml.adjoint(quantum_feature_map)(x2, wires=range(n_qubits))
        return qml.probs(wires=range(n_qubits))
    
    probs = kernel_circuit(x1, x2)
    return probs[0]

def compute_kernel_matrix(X1, X2, n_qubits, desc=""):
    """Compute kernel matrix."""
    n1, n2 = len(X1), len(X2)
    K = np.zeros((n1, n2))
    
    print(f"Computing {desc} kernel matrix ({n1}×{n2})...")
    for i in range(n1):
        if i % 10 == 0:
            print(f"  Progress: {i*n2}/{n1*n2} ({100*i/n1:.0f}%)")
        for j in range(n2):
            K[i, j] = quantum_kernel(X1[i], X2[j], n_qubits)
    print(f"  Complete: {n1*n2}/{n1*n2} (100%)\n")
    
    return K

print("\nQuantum Kernel SVM\n")
print("-" * 60)

start_time = time.time()

# Compute kernels
K_train = compute_kernel_matrix(X_train_scaled, X_train_scaled, n_qubits, "training")
K_test = compute_kernel_matrix(X_test_scaled, X_train_scaled, n_qubits, "test")

kernel_time = time.time() - start_time

# Train SVM
svm_kernel = SVC(kernel='precomputed')
y_train_01 = (y_train_q + 1) // 2
svm_kernel.fit(K_train, y_train_01)

# Predictions
y_test_pred_kernel = svm_kernel.predict(K_test)
kernel_test_acc = accuracy_score(y_test_01, y_test_pred_kernel)

print(f"Test Accuracy: {kernel_test_acc:.3f}")
print(f"Total time: {kernel_time:.1f}s\n")

print("Classification Report:")
print(classification_report(y_test_01, y_test_pred_kernel,
                           target_names=['Risky', 'Creditworthy']))

cm_kernel = confusion_matrix(y_test_01, y_test_pred_kernel)
print("Confusion Matrix:")
print(cm_kernel)

## Comparison Summary

In [ ]:
print("\n" + "=" * 80)
print("COMPARISON SUMMARY")
print("=" * 80)
print()
print(f"{'Method':<20} {'Accuracy':<12} {'Training Time':<15}")
print("-" * 50)
print(f"{'Classical SVM':<20} {test_acc:<12.3f} {classical_time:<15.2f}s")
print(f"{'Quantum VQC':<20} {vqc_test_acc:<12.3f} {vqc_time:<15.2f}s")
print(f"{'Quantum Kernel SVM':<20} {kernel_test_acc:<12.3f} {kernel_time:<15.2f}s")
print()
print("Notes:")
print("  • Quantum VQC: Competitive accuracy, much slower training")
print("  • Quantum Kernel: Similar accuracy, slow kernel computation")
print("  • Classical: Fast and competitive")
print("  • Small dataset - quantum advantage not expected")

## Comprehensive Visualization

In [ ]:
fig = plt.figure(figsize=(16, 10))

# 1. Training Loss
ax1 = plt.subplot(2, 3, 1)
epochs = np.arange(1, len(train_losses)+1) * 10
ax1.plot(epochs, train_losses, 'o-', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('VQC Training Loss')
ax1.grid(True, alpha=0.3)

# 2. Accuracy Curves
ax2 = plt.subplot(2, 3, 2)
ax2.plot(epochs, train_accs, 'o-', label='Train', linewidth=2)
ax2.plot(epochs, test_accs, 's-', label='Test', linewidth=2)
ax2.axhline(y=test_acc, color='red', linestyle='--', label='Classical SVM', alpha=0.7)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('VQC Training Progress')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Method Comparison
ax3 = plt.subplot(2, 3, 3)
methods = ['Classical\nSVM', 'Quantum\nVQC', 'Quantum\nKernel']
accuracies = [test_acc, vqc_test_acc, kernel_test_acc]
colors = ['red', 'green', 'blue']
bars = ax3.bar(methods, accuracies, color=colors, alpha=0.7)
ax3.set_ylabel('Test Accuracy')
ax3.set_title('Accuracy Comparison')
ax3.set_ylim([0, 1])
ax3.grid(True, alpha=0.3, axis='y')
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}', ha='center', va='bottom')

# 4. Training Time
ax4 = plt.subplot(2, 3, 4)
times = [classical_time, vqc_time, kernel_time]
bars = ax4.bar(methods, times, color=colors, alpha=0.7)
ax4.set_ylabel('Time (seconds)')
ax4.set_title('Training Time Comparison')
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3, axis='y')
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}s', ha='center', va='bottom')

# 5. Feature Space
ax5 = plt.subplot(2, 3, 5)
y_plot = (y_train_q + 1) // 2
scatter = ax5.scatter(X_train_pca[:, 0], X_train_pca[:, 1], 
                     c=y_plot, cmap='RdYlGn', alpha=0.6, s=50)
ax5.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
ax5.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
ax5.set_title('Feature Space (PCA Projection)')
plt.colorbar(scatter, ax=ax5, label='Class', ticks=[0, 1])
ax5.grid(True, alpha=0.3)

# 6. Quantum Kernel Matrix
ax6 = plt.subplot(2, 3, 6)
im = ax6.imshow(K_train, cmap='viridis', aspect='auto')
ax6.set_xlabel('Training Sample')
ax6.set_ylabel('Training Sample')
ax6.set_title('Quantum Kernel Matrix')
plt.colorbar(im, ax=ax6, label='Kernel Value')

plt.tight_layout()
plt.savefig('credit_classification_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Visualization complete")

## Key Takeaways

### What We Learned
✅ Quantum ML methods competitive but not superior for small data  
✅ VQC shows promise with proper hyperparameter tuning  
✅ Quantum kernels provide alternative approach to feature mapping  

### Challenges
❌ Training time significantly higher than classical  
❌ Barren plateau problem in deep circuits  
❌ Quantum advantage likely only for larger, structured datasets  

### When to Use Quantum ML
- High-dimensional data (>10 features)
- Complex feature interactions
- When quantum kernels express patterns classical kernels miss
- Research and algorithm development

### Next Steps
- Try more features (6-8)
- Experiment with different feature maps
- Test on imbalanced data
- Implement cross-validation
- Try on real credit data